# Lec17 `#14/11`

## Scheme OOP

- Basic Objects
- Inheritance
- Multiple Inheritance

### Class Diagram

Class: 
<table>
<th>Person</th>
<th>Class Name</th>
<tr><td>fname: <br />lname: </td><td>Fields</td></tr>
<tr><td>SAY <br />WHOAREYOU? </td><td>Methods</td></tr>
</table>

Instance:
<table>
<th>Person</th>
<tr><td>fname: Deniz <br />lname: Yuret </td></tr>
</table>

### Calling Convention

```scheme
(ask <obj> <method> <args> ...)
; e.g.
(define dy (make-person 'deniz 'yuret))
(ask dy 'whoareyou) ; => deniz
(ask dy 'sat '(hello world)) ; => hello world
```


### Inheritance

<table>
<th>Person</th>
<tr><td>fname: <br />lname: </td></tr>
<tr><td>SAY <br />WHOAREYOU? </td></tr>
</table>
$$ \uparrow $$
<table>
<th>Professor</th>
<tr></tr>
<tr><td>LECTURE <br />WHOAREYOU? </td></tr>
</table>
$$ \uparrow $$
<table>
<th>Arrogant Professor</th>
<tr></tr>
<tr><td>SAY</td></tr>
</table>

```scheme
(define e (make-prof 'eric 'grimson))
(ask e 'say '(the sky is blue)) ; => the sky is blue
(ask e 'WHOAREYOU?) ; => Professor grimson
(ask e 'LECTURE '(the sky is blue))
; => Therefore the sky is blue
```
```scheme
(define ap (make-arrogant-prof 'Hal 'Abelson))
(ask ap 'SAY '(the sky is blue))
; => the sky is blue obviously
```

**Question: ** What happens if
```scheme
(ask ap 'LECTURE '(the sky is blue))
```
will we get `obviously` or not ?

**Question: ** How to implement `this`, `super`, etc.

### Multiple Inheritance

<table>
<th>Singer</th>
<tr></tr>
<tr><td>SAY <br/> SING</td></tr>
</table>

$$ \uparrow $$
<table>
<th>Singing Arrogant Professor</th>
<tr></tr>
<tr></tr>
</table>

$$ \downarrow $$
<table>
<th>Arrogant Professor</th>
<tr></tr>
<tr><td>SAY</td></tr>
</table>
$$ \downarrow $$
$$ \text{...} $$


In [30]:
(define (make-person fname lname)
  (lambda (message)
    (cond ((eq? message 'WHOAREYOU) (lambda () fname))
          ((eq? message 'CHANGE-MY-NAME)
           (lambda (new-name) (set! fname new-name)))
          ((eq? message 'SAY)
           (lambda (list-of-stuff)
             (display list-of-stuff)
             'NUF-SAID))
          (else (no-method)))))

In [2]:
(define (ask obj method . args)
  (apply (obj method) args))

In [9]:
(define dy (make-person 'deniz 'yuret))
(display (ask dy 'WHOAREYOU)) (newline)
(ask dy 'CHANGE-MY-NAME 'deniz2)
(display (ask dy 'WHOAREYOU)) (newline)

(ask dy 'SAY '(the sky is blue))

deniz
deniz2
(the sky is blue)

NUF-SAID

In [21]:
(define (get-method message object)
  (object message))

(define (ask object message . args)
  (let ((method (get-method message object)))
    (cond ((method? method) (apply method args))
          (else (error "No method for message" message)))))

In [23]:
(define (method? method) #t)

(define dy (make-person 'deniz 'yuret))
(display (ask dy 'WHOAREYOU)) (newline)
(ask dy 'CHANGE-MY-NAME 'deniz2)
(display (ask dy 'WHOAREYOU)) (newline)

(ask dy 'SAY '(the sky is blue))

deniz
deniz2
(the sky is blue)

NUF-SAID

In [31]:
(define (method? x) 
  (cond ((procedure? x) #t)
        ((eq? x (no-method)) #f)
        (else (error "Object returned non-message" x))))
(define (no-method) `NO-METHOD)

(define dy (make-person 'deniz 'yuret))
(display (ask dy 'WHOAREYOU)) (newline)
(ask dy 'CHANGE-MY-NAME 'deniz2)
(display (ask dy 'WHOAREYOU)) (newline)

(ask dy 'SAY '(the sky is blue)) (newline)

(ask dy 'SOME-NON-EXISTING-METHOD)

deniz
deniz2
(the sky is blue)



Traceback (most recent call last):
  File "In [31]", line 14, col 1, in 'ask'
  File "In [21]", line 5, col 3, in 'let'
  File "In [21]", line 7, col 17, in 'error'
  File "In [21]", line 7, col 17
RunTimeError: Error in 'No method for message': 



### `self`

In [42]:
(define (make-person fname lname)
  (lambda (message)
    (cond ((eq? message 'WHOAREYOU) (lambda (self) fname))
          ((eq? message 'CHANGE-MY-NAME)
           (lambda (self new-name) 
             (set! fname new-name)
             (ask self 'SAY (list 'Call 'me fname))))
          ((eq? message 'SAY)
           (lambda (self list-of-stuff)
             (display list-of-stuff)
             'NUF-SAID))
          (else (no-method)))))

(define (ask object message . args)
  (let ((method (get-method message object)))
    (cond ((method? method) (apply method (cons object args)))
          (else (error "No method for message" message)))))

(define dy (make-person 'deniz 'yuret))
(ask dy 'WHOAREYOU)

deniz

In [43]:
(ask dy 'CHANGE-MY-NAME 'deniz2)

(Call me deniz2)

NUF-SAID

In [44]:
(ask dy 'WHOAREYOU)

deniz2

### Type

In [50]:
(define (make-person fname lname)
  (lambda (message)
    (cond ((eq? message 'WHOAREYOU) (lambda (self) fname))
          ((eq? message 'CHANGE-MY-NAME)
           (lambda (self new-name) 
             (set! fname new-name)
             (ask self 'SAY (list 'Call 'me fname))))
          ((eq? message 'SAY)
           (lambda (self list-of-stuff)
             (display list-of-stuff)
             'NUF-SAID))
          ((eq? message 'PERSON?)
           (lambda (self) #t))
          (else (no-method)))))

; Without this, ask someone 'projessor will give an error instead of #f
(define (is-a object type-pred)
  (cond ((not (procedure? object)) #f)
        (else 
         (let ((method (get-method type-pred object)))
           (cond ((method? method) (method))
                 (else #f))))))

(define dy (make-person 'deniz 'yuret))
(display (is-a dy 'PERSON?)) (newline)
(display (is-a dy 'PROFESSOR?)) (newline)


True
False
